In [1]:
import json
import os
import pandas as pd


In [2]:
directory = r'../../../data_analysis/new_2024-09-20/outdir/scale_result.xlsx'

In [3]:
mht_result = pd.read_excel(directory,sheet_name='心理健康诊断测验-MHT（9-18岁）')
insomnia_result = pd.read_excel(directory,sheet_name='阿森斯失眠量表')
watch_results = pd.read_excel(directory,sheet_name='watch_result')
phq_result = pd.read_excel(directory, sheet_name='抑郁症筛查量表PHQ-9')
pss_result = pd.read_excel(directory,sheet_name = '儿童感知压力量表（PSS-C）')

In [4]:
# Changing date of birth into age 
mht_result['birth'] = pd.to_datetime(mht_result['birth'])
phq_result['birth'] = pd.to_datetime(phq_result['birth'])
pss_result['birth'] = pd.to_datetime(pss_result['birth'])
insomnia_result['birth'] = pd.to_datetime(pss_result['birth'])
mht_result['age'] = mht_result['birth'].dt.year.map({2012: 12, 2011: 13})
phq_result['age'] = phq_result['birth'].dt.year.map({2012: 12, 2011: 13})
pss_result['age'] = pss_result['birth'].dt.year.map({2012: 12, 2011: 13})
insomnia_result['age'] = insomnia_result['birth'].dt.year.map({2012: 12, 2011: 13})

# Dropping the Unavailable users

In [5]:
watch_results

,user_id,user_name,birth,class,watch_result_Infos
0,2.537094e+38,范子涵,2011-01-20,遂昌民族中学/2024/1班,"[{'startTime': '2024-09-12T15:32:06.979', 'end..."
1,1.477914e+38,曹宇杰,2012-02-27,遂昌民族中学/2024/1班,"[{'startTime': '2024-09-12T15:32:24.756', 'end..."
2,1.103556e+38,龚嘉悦,2012-02-04,遂昌民族中学/2024/1班,"[{'startTime': '2024-09-12T15:31:40.441', 'end..."
3,1.402538e+38,潘凯莉,2011-10-05,遂昌民族中学/2024/1班,"[{'startTime': '2024-09-12T15:31:32.358', 'end..."
4,4.106441e+37,张晨熙,2012-06-29,遂昌民族中学/2024/1班,"[{'startTime': '2024-09-12T15:32:03.027', 'end..."
...,...,...,...,...,...
504,6.156942e+37,李震洋,2012-02-24,遂昌民族中学/2024/14班,"[{'startTime': '2024-09-12T14:35:59.069', 'end..."
505,2.194261e+38,徐煜棠,2012-09-20,遂昌民族中学/2024/14班,"[{'startTime': '2024-09-12T14:41:13.397', 'end..."
506,2.930861e+38,雷文慧,2012-02-08,遂昌民族中学/2024/14班,"[{'startTime': '2024-09-12T14:36:47.068', 'end..."
507,2.988898e+38,王宇航,2011-03-12,遂昌民族中学/2024/14班,"[{'startTime': None, 'endTime': None, 'dataRes..."


In [6]:
all_users = []
for result in range(len(watch_results)):
    watch_data = watch_results.iloc[result]
    student_name = watch_data['user_id']
    student_watch_info =watch_data['watch_result_Infos']
    data = str(student_watch_info).split()
    dataResultId = data[-1].split('}]')
    if dataResultId!= None:
        all_users.append((student_name, dataResultId[0]))
selected_users = [i[0] for i in all_users if i[1]!= "None"]
selected_users_dataResultId = [(i[0],i[1]) for i in all_users if i[1]!= "None"]
print(selected_users)


[2.537093913537809e+38, 1.477913642443515e+38, 1.103555664857309e+38, 1.402537984585597e+38, 4.106440794533578e+37, 2.503545104506297e+38, 3.66821449892917e+37, 2.910257551164351e+38, 2.241686246836668e+38, 2.702053694040114e+38, 2.535407591489115e+38, 4.015371756066878e+37, 1.608206718771619e+38, 1.506633766343997e+38, 1.547159297343372e+38, 2.178335927452169e+38, 3.061409590343552e+38, 2.226195553689302e+38, 2.041617940508707e+38, 2.1281112033231e+38, 2.995592229568458e+37, 2.284749799817737e+38, 2.084216281465097e+38, 2.049108347386047e+38, 8.145997167683457e+37, 1.557471075944643e+38, 2.815277054522703e+38, 1.055231843780788e+38, 9.119750029803594e+37, 2.493925693361653e+37, 1.783885570809238e+38, 2.137069020154083e+38, 7.834605180236473e+37, 3.107630916784254e+37, 5.857368778508965e+37, 2.566833142344762e+38, 6.085398598564753e+37, 1.304909481114316e+38, 1.778216023889086e+38, 1.711319608499946e+38, 1.531085960555335e+38, 2.445833261924319e+37, 2.981028409440889e+38, 2.47831054309

In [7]:
# Selected valid subjects
selected_users_scale = mht_result[mht_result['user_id'].isin(selected_users)]

user_df = pd.DataFrame(selected_users_dataResultId, columns=['user_id', 'subjectid'])
user_df['subjectid'] = user_df['subjectid'].str.strip("'")
selected_users_scale = user_df.merge(selected_users_scale, how = 'inner', on = 'user_id')

# Selecting the students whose attentiveness is good i.e. less than 5
selected_valid_data = selected_users_scale[selected_users_scale['效度']<= 4]
invalid_subject_data = selected_users_scale[selected_users_scale['效度']>4]

final_selected_ids = selected_valid_data['user_id'].to_list()
final_unselected_ids = invalid_subject_data['user_id'].to_list()

In [8]:
print("VALUE COUNT FOR SERIOUSNESS CHECK: ", selected_valid_data['效度'].sort_values(ascending= False).value_counts())

VALUE COUNT FOR SERIOUSNESS CHECK:  效度
2.0    84
1.0    84
3.0    65
4.0    45
0.0    44
Name: count, dtype: int64


# MHT DEPRESSION RESULT

In [9]:
# Finading Depression Sum
map_data = {'否': 0 ,'是': 1}
depression_columns = ['你和大家在—起时，是否觉得自己是多余的人? ', '你是否经常会突然想哭? ', '你有时是否会觉得，还是死了好？ ', '你是否经常想从高的地方跳下来? ']
selected_valid_data[depression_columns] = selected_valid_data[depression_columns].apply(lambda x: x.map(map_data))
selected_valid_data['depression_sum'] = selected_valid_data[depression_columns].sum(axis=1)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_11820\1723780855.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_valid_data[depression_columns] = selected_valid_data[depression_columns].apply(lambda x: x.map(map_data))
C:\Users\Administrator\AppData\Local\Temp\ipykernel_11820\1723780855.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_valid_data['depression_sum'] = selected_valid_data[depression_columns].sum(axis=1)


In [10]:
selected_valid_data['depression_sum'].value_counts()

depression_sum
0    170
1     71
2     39
3     24
4     18
Name: count, dtype: int64

In [11]:
selected_valid_data['depression_label'] = selected_valid_data['depression_sum'].apply(lambda x: 1 if x >= 3 else (0 if x == 0 else -1))

print(selected_valid_data.shape)

(322, 319)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_11820\2322901203.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_valid_data['depression_label'] = selected_valid_data['depression_sum'].apply(lambda x: 1 if x >= 3 else (0 if x == 0 else -1))


In [12]:
selected_valid_data['depression_label'].value_counts()

depression_label
 0    170
-1    110
 1     42
Name: count, dtype: int64

# 抑郁症筛查量表PHQ-9

In [13]:
phq_result = phq_result[phq_result['user_id'].isin(final_selected_ids)]  # User Id's are selected from MHT scale as they have the information of how many students were attentive at that time
phq_result['depression_sum'] = len(phq_result)*[0]

In [14]:
depression_scale_questions = phq_result.columns[4:]
selected_questions = []
for question in depression_scale_questions:
    if str(question).startswith('question') and str(question).endswith('core'):
        selected_questions.append(question)
phq_result['depression_sum'] = phq_result[selected_questions].sum(axis = 1)

In [15]:
phq_result_depression = selected_users_scale[['user_name','subjectid']].merge(phq_result,how= 'inner', on = 'user_name')

In [16]:
phq_result_depression['depression_label'] = phq_result_depression['depression_sum'].apply(lambda x: 1 if x >= 9 else (0 if x == 0 else -1))

In [17]:
phq_result_depression['depression_label'].value_counts()

depression_label
-1    192
 0     88
 1     42
Name: count, dtype: int64

# MHT Anxiety Result

In [18]:
# Finding Anxiety Sum 
anxiety_columns = ['你晚上要睡觉时，是否总想着明天的功课? ', '你是否一听说“要考试”心里就紧张。 ', '你学习成绩不好时，是否总是提心吊胆? ', '你考试时，想不起原先掌握的知识时，是否会感到紧张不安? ', \
                   '你考试后，在没有知道成绩之前，是否总是放心不下? ', '你是否一遇到考试，就担心会考坏? ', '你在没有完成任务之前，是否总担心完不成任务？ ', '你当着大家面朗读课文时，是否总是怕读错? ',
                   '你是否认为学校里得到的学习成绩总是不大可靠? ', '你是否认为你比别人更担心学习? ', '你是否做过考试考坏了的梦? ', '你是否做过学习成绩不好时，受到爸爸妈妈或老师训斥的梦? ', '你在工作或学习时，如果有人在注意你，你心里是否会紧张? ', \
                   '你受到老师批评时，心里是否总是不安? ', '你心里是否总觉得好像有什么事没有做好? ', '你是否总担心会发生什么意外的事？ ', '你被老师提问时，心里是否总是很紧张? ', '你是否经常咬指甲? ', '你是否经常感到呼吸困难? ', '你是否会经常急躁得坐立不安? ']
print(f"anxiety item count: {len(anxiety_columns)}")

anxiety item count: 20


In [19]:
map_data = {'否': 0 ,'是': 1}
selected_valid_data[anxiety_columns] = selected_valid_data[anxiety_columns].apply(lambda x: x.map(map_data))
selected_valid_data['anxiety_sum'] = selected_valid_data[anxiety_columns].sum(axis=1)

invalid_subject_data[anxiety_columns] = invalid_subject_data[anxiety_columns].apply(lambda x: x.map(map_data))
invalid_subject_data['anxiety_sum'] = invalid_subject_data[anxiety_columns].sum(axis = 1)
unselected_valid_data = selected_valid_data.copy()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_11820\1593359991.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_valid_data[anxiety_columns] = selected_valid_data[anxiety_columns].apply(lambda x: x.map(map_data))
C:\Users\Administrator\AppData\Local\Temp\ipykernel_11820\1593359991.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_valid_data['anxiety_sum'] = selected_valid_data[anxiety_columns].sum(axis=1)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_11820\1593359991.py:5: 

In [20]:
selected_valid_data['anxiety_label'] = selected_valid_data['anxiety_sum'].apply(lambda x: 1 if x >= 15 else (0 if x < 6 else -1))

print(selected_valid_data.shape)

(322, 321)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_11820\1578320787.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_valid_data['anxiety_label'] = selected_valid_data['anxiety_sum'].apply(lambda x: 1 if x >= 15 else (0 if x < 6 else -1))


In [21]:
selected_valid_data['anxiety_label'].value_counts()

anxiety_label
-1    205
 0     80
 1     37
Name: count, dtype: int64

# Invalid Depression Subjects

In [22]:
# Finading Depression Sum
map_data = {'否': 0 ,'是': 1}
depression_columns = ['你和大家在—起时，是否觉得自己是多余的人? ', '你是否经常会突然想哭? ', '你有时是否会觉得，还是死了好？ ', '你是否经常想从高的地方跳下来? ']
invalid_subject_data[depression_columns] = invalid_subject_data[depression_columns].apply(lambda x: x.map(map_data))
invalid_subject_data['depression_sum'] = invalid_subject_data[depression_columns].sum(axis=1)


invalid_subject_data['depression_sum'].value_counts()

invalid_subject_data['depression_label'] = invalid_subject_data['depression_sum'].apply(lambda x: 1 if x >= 3 else (0))

unselected_valid_data['depression_label'] = unselected_valid_data['depression_sum'].apply(lambda x: 0 if x == 2 or x == 1 else (-1))
final_evaluation_subjects = pd.concat([invalid_subject_data,unselected_valid_data])
final_evaluation_subjects =final_evaluation_subjects[final_evaluation_subjects['depression_label'] == 0]
print(final_evaluation_subjects.shape)

final_evaluation_subjects['depression_label'].value_counts()

(128, 320)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_11820\291713085.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invalid_subject_data[depression_columns] = invalid_subject_data[depression_columns].apply(lambda x: x.map(map_data))
C:\Users\Administrator\AppData\Local\Temp\ipykernel_11820\291713085.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invalid_subject_data['depression_sum'] = invalid_subject_data[depression_columns].sum(axis=1)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_11820\291

depression_label
0    128
Name: count, dtype: int64

# Stress Result

In [23]:
pss_result = pss_result[pss_result['user_id'].isin(final_selected_ids)]
pss_result = selected_users_scale[['user_id','subjectid']].merge(pss_result)
positive_questions = ['question_1_answer_core', 'question_4_answer_core', 'question_8_answer_core', 'question_10_answer_core', 'question_17_answer_core']
for col in positive_questions:
    pss_result[col] = 5 - pss_result[col]


pss_columns = []
for features in pss_result.columns:
    if str(features).startswith('question') and str(features).endswith('core'):
        pss_columns.append(features)


pss_result['Stress_result'] = pss_result[pss_columns].sum(axis = 1)

## Insomnia Result

In [24]:
insomnia_result = insomnia_result[insomnia_result['user_id'].isin(final_selected_ids)]
insomnia_result = selected_users_scale[['user_id','subjectid']].merge(insomnia_result)

selected_insomnia_columns = [i for i in insomnia_result.columns  if str(i).startswith('question') and str(i).endswith('core')]
insomnia_result['insomnia_result'] = insomnia_result[selected_insomnia_columns].sum(axis=1)

In [25]:
insomnia_result['insomnia_result'].value_counts().sort_index(ascending= True)

insomnia_result
0     19
1     28
2     37
3     34
4     35
5     33
6     50
7     27
8     17
9      7
10     5
11     8
12     6
13     3
14     4
15     2
16     1
19     1
21     1
24     1
Name: count, dtype: int64

# Saving The data

In [26]:
# Create a writer object and specify the Excel file path
with pd.ExcelWriter('./outdir/MiddleSchool_Depression_Anxiety.xlsx') as writer:
    phq_result_depression.to_excel(writer, sheet_name='PHQ', index=False)
    selected_valid_data.to_excel(writer, sheet_name='MHT', index=False)
    final_evaluation_subjects.to_excel(writer, sheet_name='INMHT', index=False)
    pss_result.to_excel(writer, sheet_name = 'PSS', index = False)
    insomnia_result.to_excel(writer, sheet_name= 'Insomnia', index = False )
